<a href="https://colab.research.google.com/github/sophiamaria05/IC_MDA/blob/update-code-to-first-official-version/xml_to_plantuml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparing

In [ ]:
#@title Defines break and tab for writing the puml file
_break = """
"""
_tab = '    '

##Importing libraries

In [ ]:
#reading xml files
import xml.etree.cElementTree as ET
#importing files
import os

##Defining methods

In [ ]:
#@title ***get_xml_files(xml_file_folder):*** Geting the xml files (of the predicted phrases)
def get_xml_files(xml_file_folder):
  """
    Gets the folder path where the xml file with predicted phrases.
    Returns a List[xml_files].
  """
  return [file for file in os.listdir(xml_file_folder) if file.endswith('.xml')]

In [ ]:
#@title ***tag_to_array(tag):*** Converts phrase's ***type*** to array to make checking each part of the tag easier
def tag_to_array(tag):
  tag_array = [int(tag_part) for tag_part in tag.split(".")]#gets each part of the phrase's type and saves it into tag_array
  while len(tag_array) < 3:#if the type is 0 or 0.1 it appends 0s to the tag for the type of noun and reserved verb check
    tag_array.append(0)
  return tag_array#returs the array of the particioned type tag

In [ ]:
#@title ***get_classes_and_associations(root_xml):*** Creates two dictionaries with the classes and it's methods, attributes and associations.
def get_classes_and_associations(root_xml):
  """
    get_classes_and_associations(root_xml) -> [methods_and_attributes, associations]

    Creates two dictionaries of the classes, one with it's methods and attributes, and the other with it's associations.

    Args:
      root_xml: The predicted xml root.

    Returns:
      A Tuple of two dictionaries.

        methods_and_attributes: {class: [methods, attributes]}
          A dictionary with the classes and it's methods and attributes.

        associations: {class: [classes_associated]}
          A dictionary with the classes and it's associations.
  """
  # Defines the directories:
  methods_and_attributes = {}# For the methods and attributes of each class
  associations = {}# For the associations bettween classes

  for use_case in root_xml:
    for flow in use_case:#  Goes through each use case in the xml file

      use_case_name = "_".join(use_case.text.split(" "))
      inputs = []#  List of inputs for the "Insere" case

      for phrase in flow:# Goes through each phrase of the main flow and the alternatives flows
        nodes = [node.text for node in phrase.iter()]

        while None in nodes:# Removes all Nones from the phrase nodes
          nodes.pop(nodes.index(None))

        if phrase.text in nodes:# If text of the whole phrase is in the nodes array
          nodes.pop(nodes.index(phrase.text))# Pops the phrase.text

        try:
          tag = tag_to_array(phrase.get("type"))# Converts the type tag to array for easier manipulation
        except:
          tag = [0]#Set tag as invalid phrase type

        if tag[0]==0:# If invalid phrase type
          inputs = []# Cleans the inputs so it doesnt get carried
          continue


        try:# Gets the text of the last node for the name of the class
          class_name = nodes[-1].text
        except Exception as e:
          class_name = nodes[-1]


        if tag[2]==1:#  Special cases: "informa"(inform) and "insere"(insert)
          if (phrase.find("verb").text=="Informar" or phrase.find("verb").text=="Informa") and tag[1]==1:# Inform case
            method_name = "Informa(msg)"
            try:
              methods_and_attributes["V_"+use_case_name+"  <<view>>"].append(method_name)# Appends the name of the method to the dict of the use case view class
            except:# Excepts when the use case view class wasn't already created
              methods_and_attributes["V_"+use_case_name+"  <<view>>"]=[method_name]# And defines the Informa(msg) as the first method of the class

          elif (phrase.find("verb").text=="Inserir" or phrase.find("verb").text=="Insere"):# Insert case
            inputs = nodes[2:]# Gets all words of the phrase except for the noun and verb

            attributes = inputs
            for attribute_name in attributes:#Inserts all the atributes                   ***
              try:
                methods_and_attributes[class_name].append(attribute_name)
              except:
                methods_and_attributes[class_name]=[attribute_name]


        elif tag[1]==0:#  Noun is DIFFERENT from System
          method_name = ("_".join([phrase.find("verb").text, phrase.find("object").text]))+"()"# Gets the name of the method as verb_object

          try:
            if not (method_name in methods_and_attributes["V_"+use_case_name]):
              methods_and_attributes["V_"+use_case_name+"  <<view>>"].append(method_name)# Appends the name of the method to the dict of the use case view class
          except:# Excepts when the use case view class wasn't already created
            methods_and_attributes["V_"+use_case_name+"  <<view>>"]=[method_name]# And defines this method as the first one of the class

          try:#Does the same for the use case controller class
            if not (method_name in methods_and_attributes["C_"+use_case_name]):
              methods_and_attributes["C_"+use_case_name+"  <<controller>>"].append(method_name)
          except:
            methods_and_attributes["C_"+use_case_name+"  <<controller>>"]=[method_name]


          try:#                                                                                                 ASSOCIATION TEST *****
            if not ("V_"+use_case_name in associations["C_"+use_case_name]):
              associations["C_"+use_case_name].append("V_"+use_case_name)
          except:
            associations["C_"+use_case_name]=["V_"+use_case_name]

          if not (class_name in associations["C_"+use_case_name]):
            associations["C_"+use_case_name].append(class_name)


        else:#  Noun is System
          method_name = ("_".join([phrase.find("verb").text, phrase.find("object").text]))# Gets the name of the method as verb_object

          if inputs!=[]:#                                                                                       INSERE TEST *****
            method_name+=("("+",".join(inputs))+")"
            inputs = []
          else:
            method_name+="()"

          try:
            if not (method_name in methods_and_attributes[class_name]):
              methods_and_attributes[class_name].append(method_name)
          except:
            methods_and_attributes[class_name]=[method_name]

          try:
            if not (class_name in associations["C_"+use_case_name]):
              associations["C_"+use_case_name].append(class_name)
          except:
            associations["C_"+use_case_name]=[class_name]


  return methods_and_attributes, associations

In [ ]:
#@title ***write_class_diagram(classes_and_methods):*** Writes the puml file using the classes dictionary created
def write_class_diagram(classes_and_methods, associations):
  plantuml = """@startuml classDiagram

<style>
classDiagram{
    class{
        .view{
            BackgroundColor lightgreen
        }
    }
}
</style>
hide <<view>> stereotype
"""

  for class_name in classes_and_methods:
    plantuml += _break+"class "+class_name+"{"
    for method in classes_and_methods[class_name]:
      plantuml += _break+_tab+method
    plantuml += _break+"}"+_break
  plantuml += _break
  for association in associations:#                                                                             ASSOCIATION TEST *****
    for class_name in associations[association]:
      plantuml += _break+association+" -- "+class_name+_break
  plantuml += _break+"@enduml"
  return plantuml

In [ ]:
#@title ***name_file(save_folder, xml_file):*** Defines the name of the written file
def name_file(save_folder, xml_file):
  if '.xml' == xml_file[-4:]:
    xml_file = xml_file[:-4]
  if "predicted_phrases_types (" == xml_file[:19] and ")" == xml_file[-1:]:
    return save_folder+'class_diagram ('+xml_file[19:-1]+')'
  else:
    return save_folder+'class_diagram ('+xml_file+')'

#Main method

In [ ]:
#@title ***get_pumls(xml_file_folder)***
def get_pumls(xml_file_folder, save_folder):
  pumls = []
  i=0
  for xml_file in get_xml_files(xml_file_folder):
    print("xml_file: ", xml_file_folder+xml_file)
    root_xml = ET.parse(xml_file_folder+xml_file).getroot()
    classes_and_methods, associations = get_classes_and_associations(root_xml)
    puml = write_class_diagram(classes_and_methods, associations)
    print("\n")
    puml_name = name_file(save_folder, xml_file)
    if i==0:
      puml_name += '.puml'
    else:
      puml_name += '('+ str(i) +').puml'
    i+=1
    with open(puml_name, 'w') as file:
      file.write(puml)
    pumls.append({'puml_name': puml_name, 'puml': puml})
  return pumls

#Writing plantumls

In [ ]:
#@title Folder where the **xml files** of the phrases was uploaded
xml_file_folder = os.environ.get("PREDICTED_XML_DIR")
if xml_file_folder == None or xml_file_folder == "":
  xml_file_folder = "/content/"

In [ ]:
#@title Folder where the **puml files** are going to be saved
save_folder = os.environ.get("PUML_XML_DIR")
if save_folder == None or save_folder == "":
  save_folder = "/content/puml_files/"
os.makedirs(save_folder, exist_ok=True)

In [ ]:
#@title Genrerate and print the pumls
i = 0
for puml_dir in get_pumls(xml_file_folder, save_folder):
  print("\nPuml_"+str(i)+" -> "+puml_dir['puml_name']+":\n\n")
  i+=1
  print(puml_dir['puml'])
  print("\n\n\n")